In [0]:
# Library to process MIDI files
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv1D,MaxPooling1D, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Embedding

from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [0]:
# !pip install mido

In [0]:
import mido
from mido import MidiFile, MidiTrack, Message

In [83]:
tf.__version__

'2.2.0'

### Data Preprocessing

In [84]:
music_list = pd.read_csv('music_list.csv')
music_list = music_list.fillna('Not Available')
music_list

,1-Calming,2-Sports,3-Mysterious,4-Nostalgia,5-Happy
0,Big Sky,Shiatsu,Vague,All That You Will Be,Shadows in Vienna
1,Maybe Another Day,Kermode,Cloak,Fractured Memories,Magic Forest
2,The Human Derivative,Run Till Dawn,Lethal Secrets,When It Rains It Pours,Mistaken by Accident
3,The Crossing,Whatever Man,Remote,Near Her,New Orleans Diner
4,A Presence Felt,Body Work,The Mole,Days,Life in Tinseltown
5,No Good Byes,Space Palace,The Unveiling,Mornings,Harlem Rollout
6,Awash,Black Coffee,Ginger Root,Always the Same,Summertime Memory
7,A Whisper In The Dark,Unleash My Halo,Fiona,Through These Doors,Countryside Living
8,Whisper of the Pines,Way Way Back,Dust Blaster,Heading Home Again,On the Rooftop
9,Traces,The Sooner The Better,Fighter Jet,A Lighthouse In Space,The Upper Hand


In [0]:
# find a filename that contains a string

# import os

# folder_name = '1-Calming'
# keyword = 'A Presence Felt'
# for fname in os.listdir('midi/'+folder_name):
#     if keyword in fname:
#         print(fname, "has the keyword")

In [0]:
# prepare a dictionary for song names for each category

# import os
# song_dict = {}
# for folder_name in music_list.columns:
# #     print(folder_name, '\n')
#     song_dict[folder_name] = []
#     for fname in os.listdir('midi/'+folder_name):
# #         print('fname: ', fname)
#         song_dict[folder_name].append(fname)

# print(song_dict['1-Calming'])

In [0]:
# song_dict.keys()

In [0]:
# X = []
# y = []
# for cat in song_dict.keys():
#     for fname in song_dict[cat]:
#         if fname != '.DS_Store':
#             y.append(cat)
#     #         midi_filename = 'midi/1-calming/ES_A Presence Felt - Gavin Luke.mp3.mid'
#             midi_filename = 'midi/' + cat + '/' + fname
# #             print(midi_filename)
#             midi = MidiFile(midi_filename)
#             cnt = 0
#             for msg in midi:
#                 print(msg)
#                 cnt+=1
#                 if cnt>100:
#                     break 

#             # save notes for the song as a list
#             notes = []
#             for msg in midi:
#                 if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
#                     data = msg.bytes()
#                     notes.append(data[1]) # we are just adding the note!

#             ## rescale all the notes to the [0,1] interval
#             scaler = MinMaxScaler(feature_range=(0,1))
#             scaler.fit(np.array(notes).reshape(-1,1))
#             notes = list(scaler.transform(np.array(notes).reshape(-1,1)))
#             notes = [list(note)[0] for note in notes]

#             X.append(notes)
        
# print(X)
# print(y)


In [0]:
# df_X = pd.DataFrame(X)
# df_X.to_csv('df_X.csv', index=False)
# df_y = pd.DataFrame(y)
# df_y.to_csv('df_y.csv', index=False)

In [90]:
df_X = pd.read_csv('df_X.csv')
df_X = df_X.fillna(0)
df_X.shape

(50, 44841)

In [91]:
df_y = pd.read_csv('df_y.csv')
df_y.shape

(50, 1)

In [0]:
X_read = np.array([df_X.iloc[i,:].tolist() for i in range(df_X.shape[0])])

In [93]:
X_read.shape

(50, 44841)

In [0]:
y = df_y['0'].tolist()

In [95]:
# generate dummy variable y

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
encoded_Y
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y.shape

(50, 5)

#### Data Augmentation



In [0]:
n_col = df_X.shape[1]
len_seq = 800

In [97]:
n_col / len_seq

56.05125

In [98]:
n_col//len_seq

56

In [0]:
n_col = df_X.shape[1]
# len_seq = 1000

X_aug = []
y_aug = []

for i_row in range(df_y.shape[0]):
  song_data = df_X.iloc[i_row,:]
  # print(song_data)
  # print(song_data.shape)
  for i_seg in range((n_col//len_seq)):
    col_start = i_seg * len_seq
    # print(col_start)
    col_end = (i_seg + 1) * len_seq
    # print(col_end)
    seg = song_data.iloc[col_start:col_end]
    if sum(seg) == 0.0:
      continue
    else:
      X_aug.append(seg)
      y_aug.append(df_y.iloc[i_row])
# print(X_aug)
# print(y_aug)


In [100]:
np.shape(X_aug)

(835, 800)

In [101]:
np.shape(y_aug)

(835, 1)

In [102]:
unique, counts = np.unique(y_aug, return_counts=True)
dict(zip(unique, counts))

{'1-Calming': 69,
 '2-Sports': 303,
 '3-Mysterious': 219,
 '4-Nostalgia': 82,
 '5-Happy': 162}

In [0]:
# X_read = np.array([df_X.iloc[i,:].tolist() for i in range(df_X.shape[0])])

X_read = np.array(X_aug)


In [104]:
# generate dummy variable y

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_aug)
encoded_Y = encoder.transform(y_aug)
encoded_Y
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y.shape

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(835, 5)

### Define Parameters

In [0]:
# n_prev = 10000

n_prev = len_seq
test_size = 0.2

### Neural Network

In [0]:
# split datasets into train and test
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(X_read, dummy_y, test_size = test_size, 
                                                    random_state = 0)

In [0]:
# set the length of all sequences to be equal to max_len

max_len = 0
temp_len = 0
for i in range(len(X_read)):
    max_len = len(X_read[i])
#     print(max_len)
    if max_len < temp_len:
        max_len = temp_len
    else:
        temp_len = max_len
max_len

# X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_train_nn = X_train_nn.reshape((X_train_nn.shape[0], X_train_nn.shape[1],1))

# X_test = sequence.pad_sequences(X_test, maxlen=max_len)
X_test_nn = X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1)


#### Only LSTM + Dense Layer

In [108]:
X_train_nn.shape

(668, 800, 1)

In [109]:
# Define a model
# n_prev = 15000

model = Sequential()

model.add(LSTM(256 , input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.3))

model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'sigmoid'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

# train model
# model.fit(np.array(X_train_nn[:,:n_prev]), np.array(y_train_nn), 
#           32, 20, verbose=1)#, callbacks=[model_save_callback])

model.fit(np.array(X_train_nn), np.array(y_train_nn), 
          32, 50, verbose=1)#, callbacks=[model_save_callback])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 800, 256)          264192    
_________________________________________________________________
dropout_10 (Dropout)         (None, 800, 256)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 800, 128)          197120    
_________________________________________________________________
dropout_11 (Dropout)         (None, 800, 128)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)              

In [110]:
pred_proba = model.predict_proba(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1))

for i in range(len(y_test_nn)):
  print('Probability Prediction: ',pred_proba[i,:], '; Real Target: ', y_test_nn[i])


Probability Prediction:  [0.2398918  0.9670719  0.72586125 0.2750857  0.5498199 ] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.23998451 0.9670253  0.72581565 0.2751562  0.549827  ] ; Real Target:  [0. 0. 0. 1. 0.]
Probability Prediction:  [0.23989719 0.9670671  0.7258609  0.27509344 0.54981965] ; Real Target:  [1. 0. 0. 0. 0.]
Probability Prediction:  [0.2398889  0.9670733  0.72586274 0.27508345 0.5498196 ] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.23990977 0.9670621  0.725854   0.27510035 0.5498211 ] ; Real Target:  [0. 0. 1. 0. 0.]
Probability Prediction:  [0.23990354 0.96706593 0.72585624 0.2750944  0.54982096] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.23995051 0.96704364 0.7258316  0.27512783 0.54982483] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [0.23990986 0.96706295 0.7258527  0.27509886 0.5498215 ] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [0.23988742 0.9670751  0.72586286 0.27508008 0.5498199 

In [111]:
# X_test_trunc = X_test_nn[:, :n_prev]
# scores = model.evaluate(X_test_trunc.reshape(X_test_trunc.shape[0], X_test_trunc.shape[1], 1),
#                         y_test_nn, verbose=1)


scores = model.evaluate(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1),
                        y_test_nn, verbose=1)

print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

6/6 [==============================] - 5s 882ms/step - loss: 1.4738 - accuracy: 0.3772
Out-of-Sample Accuracy: 37.72%


#### Conv1d + LSTM + Dense Layers

In [112]:
# Define a model
n_prev = 15000
embedding_vector_length = 300
# max_len = 1000

model = Sequential()
# model.add(Embedding(n_prev, embedding_vector_length, input_length=max_len))

model.add(Conv1D(filters=256, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))

model.add(LSTM(256 , input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.3))


model.add(Conv1D(filters=128, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))

model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.3))

model.add(Conv1D(filters=64, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))

model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.3))

# model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
# model.add(Dropout(0.3))

model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'sigmoid'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

# train model
# model.fit(np.array(X_train_nn[:,:n_prev]), np.array(y_train_nn), 
#           32, 20, verbose=1)#, callbacks=[model_save_callback])

model.fit(np.array(X_train_nn), np.array(y_train_nn), 
          32, 50, verbose=1)#, callbacks=[model_save_callback])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 14993, 256)        2304      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 3748, 256)         0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 3748, 256)         525312    
_________________________________________________________________
dropout_13 (Dropout)         (None, 3748, 256)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 3741, 128)         262272    
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 935, 128)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 935, 128)         

In [113]:
# X_test_trunc = X_test_nn[:, :n_prev]
# scores = model.evaluate(X_test_trunc.reshape(X_test_trunc.shape[0], X_test_trunc.shape[1], 1),
#                         y_test_nn, verbose=1)
# print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

pred_proba = model.predict_proba(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1))

for i in range(len(y_test_nn)):
  print('Probability Prediction: ',pred_proba[i,:], '; Real Target: ', y_test_nn[i])
  
scores = model.evaluate(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1),
                        y_test_nn, verbose=1)

print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

Probability Prediction:  [0.21694732 0.840844   0.61435443 0.22612372 0.45573324] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.21696058 0.8408377  0.61434555 0.22612947 0.4557358 ] ; Real Target:  [0. 0. 0. 1. 0.]
Probability Prediction:  [0.21702203 0.8408078  0.61431265 0.2261531  0.4557457 ] ; Real Target:  [1. 0. 0. 0. 0.]
Probability Prediction:  [0.21698645 0.84082395 0.6143321  0.22614077 0.45574257] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.21698442 0.84082484 0.6143341  0.22613978 0.45574015] ; Real Target:  [0. 0. 1. 0. 0.]
Probability Prediction:  [0.21695298 0.8408407  0.61435145 0.22612649 0.4557345 ] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.21695402 0.84084094 0.6143508  0.2261262  0.45573354] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [0.21693885 0.8408485  0.6143593  0.22611982 0.45573092] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [0.21693537 0.84084976 0.6143619  0.22611877 0.4557303 

#### Define a different model with more focus on using Conv1D

In [114]:
# Define a model
n_prev = 15000
embedding_vector_length = 300
# max_len = 1000

model = Sequential()
# model.add(Embedding(n_prev, embedding_vector_length, input_length=max_len))

# model.add(Conv1D(filters=128, kernel_size=8, strides=1, activation='relu', 
#                  input_shape=(n_prev, 1)))
# model.add(MaxPooling1D(pool_size=4))

model.add(Conv1D(filters=128, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.3))


model.add(Conv1D(filters=64, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.3))

model.add(Conv1D(filters=32, kernel_size=8, strides=1, activation='relu', 
                 input_shape=(n_prev, 1)))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.3))

model.add(LSTM(16, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.3))
# model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'sigmoid'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

# train model
# model.fit(np.array(X_train_nn[:,:n_prev]), np.array(y_train_nn), 
#           32, 20, verbose=1)#, callbacks=[model_save_callback])
model.fit(np.array(X_train_nn), np.array(y_train_nn), 
          32, 50, verbose=1)#, callbacks=[model_save_callback])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, 14993, 128)        1152      
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 3748, 128)         0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 3748, 128)         0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 3741, 64)          65600     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 935, 64)           0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 935, 64)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 928, 32)          

In [115]:
# X_test_trunc = X_test_nn[:, :n_prev]
# scores = model.evaluate(X_test_trunc.reshape(X_test_trunc.shape[0], X_test_trunc.shape[1], 1),
#                         y_test_nn, verbose=0)
# print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

pred_proba = model.predict_proba(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1))

for i in range(len(y_test_nn)):
  print('Probability Prediction: ',pred_proba[i,:], '; Real Target: ', y_test_nn[i])
  
scores = model.evaluate(X_test_nn.reshape(X_test_nn.shape[0], X_test_nn.shape[1], 1),
                        y_test_nn, verbose=1)

print("Out-of-Sample Accuracy: %.2f%%" % (scores[1]*100))

Probability Prediction:  [0.00177243 0.13461673 0.04494357 0.0176014  0.3294388 ] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.0042429  0.44632894 0.01893416 0.01955339 0.3761583 ] ; Real Target:  [0. 0. 0. 1. 0.]
Probability Prediction:  [0.84383583 0.06300849 0.6007195  0.11348075 0.01916498] ; Real Target:  [1. 0. 0. 0. 0.]
Probability Prediction:  [0.6647524  0.10382482 0.32443684 0.2871435  0.29884112] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [0.8289955  0.08060288 0.5739329  0.11857861 0.02758509] ; Real Target:  [0. 0. 1. 0. 0.]
Probability Prediction:  [0.00306454 0.11473164 0.03172457 0.02648404 0.53345   ] ; Real Target:  [0. 0. 0. 0. 1.]
Probability Prediction:  [4.5301669e-05 8.3098495e-01 1.2380123e-02 1.7598271e-04 1.0161072e-02] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [3.1797255e-05 8.3660090e-01 1.0991693e-02 1.3077259e-04 8.1668496e-03] ; Real Target:  [0. 1. 0. 0. 0.]
Probability Prediction:  [3.4553832e-05 7.5581110e

### Other Classifiers

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# X = df_X.to_numpy()
# y = df_y.to_numpy()
X = X_aug
y = y_aug

In [0]:
# n_prev = 15000

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, 
                                                    random_state = 0)

X_train_trunc = X_train
X_test_trunc = X_test
# X_train_trunc = X_train[:, :n_prev]
# X_train_trunc = np.array(X_train[:,:n_prev]).reshape(X_train_trunc.shape[0], 
#                                     X_train_trunc.shape[1])
# y_train = np.array(y_train)

# X_test_trunc = X_test[:, :n_prev]
# X_test_trunc = X_test_trunc.reshape(X_test_trunc.shape[0], 
#                                     X_test_trunc.shape[1])

In [0]:
y_test = np.array(y_test)

In [0]:
### Random Forest
from sklearn.ensemble import RandomForestClassifier

In [120]:
rf_clf = RandomForestClassifier(n_estimators = 88,
                                max_depth=2, 
                                min_samples_leaf=2,
                                random_state=0, verbose=0)

param_grid = {
    'n_estimators': [8, 9, 10, 11, 12],
    'max_depth': [1, 2, 3,4],
    'min_samples_leaf': [2,4, 5, 6,7]
}

grid_rf_clf = GridSearchCV(rf_clf, param_grid, cv=3, verbose=0)
grid_rf_clf.fit(X_train_trunc, y_train)

grid_rf_pred = grid_rf_clf.predict(X_test_trunc)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

In [121]:
print('accuracy: ', accuracy_score(grid_rf_pred, y_test), '\n')
print('parameters: ', grid_rf_clf.best_params_, '\n')

print('testing set: ', y_test.reshape(y_test.shape[1], 
                                      y_test.shape[0]), '\n')
print('prediction: ', grid_rf_pred)

accuracy:  0.5209580838323353 

parameters:  {'max_depth': 4, 'min_samples_leaf': 6, 'n_estimators': 12} 

testing set:  [['5-Happy' '4-Nostalgia' '1-Calming' '5-Happy' '3-Mysterious' '5-Happy'
  '2-Sports' '2-Sports' '2-Sports' '3-Mysterious' '1-Calming' '2-Sports'
  '4-Nostalgia' '2-Sports' '3-Mysterious' '3-Mysterious' '3-Mysterious'
  '3-Mysterious' '2-Sports' '3-Mysterious' '3-Mysterious' '4-Nostalgia'
  '4-Nostalgia' '2-Sports' '5-Happy' '5-Happy' '3-Mysterious' '5-Happy'
  '4-Nostalgia' '4-Nostalgia' '3-Mysterious' '3-Mysterious' '4-Nostalgia'
  '2-Sports' '2-Sports' '2-Sports' '2-Sports' '1-Calming' '3-Mysterious'
  '3-Mysterious' '3-Mysterious' '1-Calming' '4-Nostalgia' '2-Sports'
  '1-Calming' '4-Nostalgia' '5-Happy' '4-Nostalgia' '2-Sports' '5-Happy'
  '3-Mysterious' '5-Happy' '2-Sports' '5-Happy' '3-Mysterious'
  '4-Nostalgia' '5-Happy' '3-Mysterious' '2-Sports' '2-Sports'
  '4-Nostalgia' '5-Happy' '2-Sports' '3-Mysterious' '2-Sports'
  '3-Mysterious' '3-Mysterious' '5-Happ

In [0]:
### SVM Classifier
from sklearn.svm import SVC

In [123]:
svc = SVC()

param_grid = {
    'kernel': ['poly', 'rbf', 'sigmoid','linear'],
    'C': [5, 10, 15, 30, 100],
    'gamma': [0.1, 0.001, 0.0008, 0.0005]
}

grid_svc = GridSearchCV(svc, param_grid, cv=3, verbose=0)
grid_svc.fit(X_train_trunc, y_train)

grid_svc_pred = grid_svc.predict(X_test_trunc)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [124]:
print('accuracy: ', accuracy_score(grid_svc_pred, y_test), '\n')
print('parameters: ', grid_svc.best_params_, '\n')

print('testing set: ', y_test.reshape(y_test.shape[1], 
                                      y_test.shape[0]), '\n')
print('prediction: ', grid_svc_pred)

accuracy:  0.5029940119760479 

parameters:  {'C': 5, 'gamma': 0.001, 'kernel': 'rbf'} 

testing set:  [['5-Happy' '4-Nostalgia' '1-Calming' '5-Happy' '3-Mysterious' '5-Happy'
  '2-Sports' '2-Sports' '2-Sports' '3-Mysterious' '1-Calming' '2-Sports'
  '4-Nostalgia' '2-Sports' '3-Mysterious' '3-Mysterious' '3-Mysterious'
  '3-Mysterious' '2-Sports' '3-Mysterious' '3-Mysterious' '4-Nostalgia'
  '4-Nostalgia' '2-Sports' '5-Happy' '5-Happy' '3-Mysterious' '5-Happy'
  '4-Nostalgia' '4-Nostalgia' '3-Mysterious' '3-Mysterious' '4-Nostalgia'
  '2-Sports' '2-Sports' '2-Sports' '2-Sports' '1-Calming' '3-Mysterious'
  '3-Mysterious' '3-Mysterious' '1-Calming' '4-Nostalgia' '2-Sports'
  '1-Calming' '4-Nostalgia' '5-Happy' '4-Nostalgia' '2-Sports' '5-Happy'
  '3-Mysterious' '5-Happy' '2-Sports' '5-Happy' '3-Mysterious'
  '4-Nostalgia' '5-Happy' '3-Mysterious' '2-Sports' '2-Sports'
  '4-Nostalgia' '5-Happy' '2-Sports' '3-Mysterious' '2-Sports'
  '3-Mysterious' '3-Mysterious' '5-Happy' '2-Sports' '4-N

In [0]:
### Nearest neighbors
from sklearn.neighbors import KNeighborsClassifier

In [126]:
neigh = KNeighborsClassifier()
param_grid = {'n_neighbors': [1,2, 3,5,7]}
grid_neigh = GridSearchCV(neigh, param_grid, cv=3, verbose=0)

grid_neigh = grid_neigh.fit(X_train_trunc, y_train)
grid_neigh_pred = grid_neigh.predict(X_test_trunc)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when

In [127]:
print('accuracy: ', accuracy_score(grid_neigh_pred, y_test), '\n')
print('parameters: ', grid_neigh.best_params_, '\n')

accuracy:  0.2634730538922156 

parameters:  {'n_neighbors': 2} 



In [0]:
### Multivariate Logistic regression
from sklearn.linear_model import LogisticRegression

In [129]:
logit = LogisticRegression()
param_grid = {
    'penalty': ['l2', 'l1', 'elasticnet', 'none'],
    'C': [0.01, 0.1, 0.5, 1, 2]
}

grid_logit = GridSearchCV(logit, param_grid, cv=3, verbose=0)
grid_logit.fit(X_train_trunc, y_train)

grid_logit_pred = grid_logit.predict(X_test_trunc)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

In [130]:
print('accuracy: ', accuracy_score(grid_logit_pred, y_test), '\n')
print('parameters: ', grid_logit.best_params_, '\n')

accuracy:  0.49700598802395207 

parameters:  {'C': 0.01, 'penalty': 'l2'} 



In [0]:
### Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
# X_train_trunc.shape
# y_train.shape

In [133]:
# gbc = GradientBoostingClassifier()
# param_grid = {
#     'loss': ['deviance', 'exponential'],
#     'learning_rate': [0.01, 0.05, 0.1, 0.5],
#     'n_estimators': [80, 100, 150, 200, 300],
#     'min_samples_leaf': [1, 2, 5, 8, 10],
#     'max_depth': [3,5,8,12,15]
# }

# grid_gbc = GridSearchCV(gbc, param_grid, cv=3, verbose=0)
# grid_gbc.fit(X_train_trunc, y_train)

# grid_gbc_pred = grid_gbc.predict(X_test_trunc)

gbc = GradientBoostingClassifier()
gbc.fit(X_train_trunc, y_train)

gbc_pred = gbc.predict(X_test_trunc)



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [134]:

print('accuracy: ', accuracy_score(gbc_pred, y_test), '\n')
# print('parameters: ', gbc.best_params_, '\n')
print('parameters: ', gbc.get_params, '\n')

accuracy:  0.5568862275449101 

parameters:  <bound method BaseEstimator.get_params of GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)> 

